In [1]:

#Importing libraries
import requests
import os
import json
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup  
import time
import requests
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import traceback
from webdriver_manager.chrome import ChromeDriverManager  


## Initializing a chrome instance

In [18]:
options = webdriver.ChromeOptions() 


In [19]:
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

In [20]:
driver = webdriver.Chrome(options=options, executable_path=r"C:/selenium/chromedriver.exe")


## Creating issue links 

In [42]:
issues=[]

In [43]:
for i in range(1,25):
    a='https://academic.oup.com/rof/issue/'+str(i)+'/'
    for j in range(1,7):
        b=a+str(j)
        issues.append(b)

## Setting up the home window

In [45]:
window0=driver.window_handles[0]

In [46]:
driver.switch_to.window(window0)

## Creating an empty list to store the data

In [37]:
rf=[]

In [ ]:
for i in issues:
    try:
        driver.get(i)
        sleep(5)
        articles = WebDriverWait(driver, 30).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, '.customLink.item-title a')))
        for article in articles:                                                                    
            
            try:  
                ActionChains(driver).key_down(Keys.CONTROL).click(article).key_up(Keys.CONTROL).perform()
                WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))
                window1 = driver.window_handles[1]
                driver.switch_to.window(window1)
                sleep(8)
                rof={}
                try:
                    rof['title']=driver.find_element_by_xpath('//*[@id="ContentColumn"]/div[2]/div[1]/div/div/h1').text
                except:
                    print("title not found")
                try:
                    rof['authors']=driver.find_element_by_xpath('//*[@id="ContentColumn"]/div[2]/div[1]/div/div/div[1]/div').text
                except:
                    print("authors not found")
                try:
                    rof['article_details']=driver.find_element_by_xpath('//*[@id="ContentColumn"]/div[2]/div[1]/div/div/div[2]/div[1]/div').text
                except:
                    print("article details not found")
                try:
                    rof['published_time']=driver.find_element_by_xpath('//*[@id="ContentColumn"]/div[2]/div[1]/div/div/div[2]/div[2]/div/div[2]').text
                except:
                    print("publsihed_time not found")
                try:
                    rof['abstract']=driver.find_element_by_xpath('//*[@id="ContentTab"]/div[1]/div/div/section').text
                except:
                    print("abstract not  found")
                try:
                    rof['jel_classification']=driver.find_element_by_xpath('//*[@id="ContentTab"]/div[1]/div/div/div[1]/div[1]/div').text
                except:
                    print("jel not found")
                try:
                    rof['associate_editor']=driver.find_element_by_xpath('//*[@id="ContentTab"]/div[1]/div/div/div[1]/div[3]/div/div/span').text
                except:
                    print("editor nit found")
                try:
                    rof['body']=driver.find_element_by_xpath('//*[@id="ContentTab"]/div[1]/div/div').text
                except:
                    print("body missing")
                rf.append(rof)
                driver.close()
                driver.switch_to.window(window0)
            except:
                print("couldnt get the article")
    except:
            print("issue not found")

## Storing the data in a dataframe

In [49]:
data=pd.DataFrame(rf)

## Feature engineering to extract more variables 

In [51]:
info1=data['body'].str.split('References\n',expand=True)

In [53]:
data['body']=info1[0]

In [54]:
data['references']=info1[1]

In [55]:
data

,title,authors,article_details,published_time,abstract,jel_classification,body,references
0,Agency Conflicts and Risk Management*,"Erwan Morellec,\nClifford W. Smith, Jr.\nAutho...","Review of Finance, Volume 11, Issue 1, 2007, P...",01 January 2007,This paper analyzes the relation between agenc...,JEL G13 - Contingent Pricing; Futures Pricing ...,Abstract\nThis paper analyzes the relation bet...,"Alderson M., Betker B.. Liquidation costs and..."
1,On the Evolution of Investment Strategies and ...,"Terje Lensberg,\nKlaus Reiner Schenk-Hoppé\nAu...","Review of Finance, Volume 11, Issue 1, 2007, P...",01 January 2007,This paper complements theoretical studies on ...,JEL C63 - Computational Techniques; Simulation...,Abstract\nThis paper complements theoretical s...,"Algoet P. H., Cover T. M.. Asymptotic optimal..."
2,Should smart investors buy funds with high pas...,"Frederic Palomino,\nHarald Uhlig\nAuthor Notes","Review of Finance, Volume 11, Issue 1, 2007, P...",01 January 2007,We fully characterize the equilibria in a gme ...,JEL G11 - Portfolio Choice; Investment Decisio...,Abstract\nWe fully characterize the equilibria...,"Berk J., Green R.. Mutual fund flows and perf..."
3,Stochastic Dominance Bounds on American Option...,"George M. Constantinides,\nStylianos Perrakis\...","Review of Finance, Volume 11, Issue 1, 2007, P...",31 January 2007,We derive equilibrium restrictions on the rang...,JEL G13 - Contingent Pricing; Futures Pricing,Abstract\nWe derive equilibrium restrictions o...,Amin K. I.. Jump diffusion option valuation in...
4,Market Responses to Buy Recommendations Issued...,"Alexander G. Kerl,\nAndreas Walter\nAuthor Notes","Review of Finance, Volume 11, Issue 1, 2007, P...",31 January 2007,This paper analyzes explicit buy recommendatio...,JEL G11 - Portfolio Choice; Investment Decisio...,Abstract\nThis paper analyzes explicit buy rec...,"Admati A. R., Pfleiderer P.. Direct and indir..."
...,...,...,...,...,...,...,...,...
603,Internal Capital Markets in Times of Crisis: T...,"Raffaele Santioni,\nFabio Schiantarelli,\nPhil...","Review of Finance, Volume 24, Issue 4, July 20...",12 September 2019,Firms affiliated with business groups survive ...,JEL G30 - General G32 - Financing Policy; Fina...,Abstract\nFirms affiliated with business group...,"Accornero M., Alessandri P., Carpinelli L., So..."
604,Margin Trading and Comovement During Crises*,"Bige Kahraman,\nHeather Tookes\nAuthor Notes","Review of Finance, Volume 24, Issue 4, July 20...",27 September 2019,We exploit threshold rules governing margin tr...,JEL G01 - Financial Crises G14 - Information a...,Abstract\nWe exploit threshold rules governing...,"Anton M., Polk C. (2014): Connected stocks, Jo..."
605,How Do Internal Capital Markets Work? Evidence...,"David Buchuk,\nBorja Larrain,\nMounu Prem,\nFr...","Review of Finance, Volume 24, Issue 4, July 20...",18 October 2019,We study the inner workings of internal capita...,JEL G32 - Financing Policy; Financial Risk and...,Abstract\nWe study the inner workings of inter...,"Abadie A., Diamond A., Hainmueller J. (2010): ..."
606,Do Corporate Governance Ratings Change Investo...,"Paul M Guest,\nMarco Nerino\nAuthor Notes","Review of Finance, Volume 24, Issue 4, July 20...",09 October 2019,This paper examines empirically the announceme...,JEL G14 - Information and Market Efficiency; E...,Abstract\nThis paper examines empirically the ...,"Alexander C., Chen M., Seppi D., Spatt C. (201..."


## Saving the data into a csv

In [56]:
data.to_csv("rof.csv")